<a href="https://colab.research.google.com/github/samibahig/AutomaticSeedDetection/blob/main/FeedForward.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
pd.set_option("max_columns", None)
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import seaborn as sns
import numpy as np
import torch
import torch.nn as nn
# Acquisition des données et Affichage
df = pd.read_csv("diabetes_012_health_indicators_BRFSS2015.csv")
print('VOICI LE CONTENU DU DATAFRAME df')
df

In [ ]:
df.loc[df['Diabetes_012'] == 2, 'Diabetes_012'] -= 1

In [ ]:
df2 = df.drop(columns = ['CholCheck', 'Income', 'Stroke','HighChol','Education', 'DiffWalk', 'NoDocbcCost', 'AnyHealthcare'], axis=1)

In [ ]:
df2

In [ ]:
df2.head()

In [ ]:
corr=df2.corr()
corr=corr.round(2)
f,ax = plt.subplots(figsize=(10,8))
sns.heatmap(corr)
plt.show()

In [ ]:
### Puisque nous avons des outliers, nous allons utilisé un Robust Scaler pour la standardisation

In [ ]:
from sklearn.preprocessing import RobustScaler
sc =  RobustScaler()
df_std = sc.fit_transform(df2)
df_std = pd.DataFrame(df_std)
df_std.head()

In [ ]:
df_std

In [ ]:
df_std.fillna(df_std.mean(), inplace=True)

In [ ]:
df_std

In [ ]:
y = df2['Diabetes_012']
X = df2.drop('Diabetes_012', axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 44)

In [ ]:
train_dataset = x_train, y_train

In [ ]:
test_dataset = x_test, y_test

In [ ]:
batch_size = 16
n_iters = 90000
#num_epochs = n_iters / (len(x_train) / batch_size)
#num_epochs = int(num_epochs)

In [ ]:
len(x_train)

In [ ]:
num_epochs

In [ ]:
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [ ]:
train_dataset

In [ ]:
training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class FeedforwardNeuralNetModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FeedforwardNeuralNetModel, self).__init__()
        # Linear function
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity
        self.sigmoid = nn.Sigmoid()
        # Linear function (readout)
        self.fc2 = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        # Linear function  # LINEAR
        out = self.fc1(x)
        # Non-linearity  # NON-LINEAR
        out = self.sigmoid(out)
        # Linear function (readout)  # LINEAR
        out = self.fc2(out)
        return out

In [ ]:
input_dim = 13
hidden_dim = 100
output_dim = 1
model = FeedforwardNeuralNetModel(input_dim, hidden_dim, output_dim)

In [ ]:
criterion = torch.nn.BCEWithLogitsLoss()

In [ ]:
learning_rate = 0.075
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 

In [ ]:
print(model.parameters())
print(len(list(model.parameters())))
# FC 1 Parameters 
print(list(model.parameters())[0].size())
# FC 1 Bias Parameters
print(list(model.parameters())[1].size())
# FC 2 Parameters
print(list(model.parameters())[2].size())
# FC 2 Bias Parameters
print(list(model.parameters())[3].size())

In [ ]:
### Pandas to numpy, and
x_train_numpy = x_train.to_numpy()
x_train_tensor = torch.from_numpy(x_train_numpy)
y_train_numpy = y_train.to_numpy()
y_train_tensor = torch.from_numpy(y_train_numpy)
print(x_train_tensor, y_train_tensor)

In [ ]:
### Pandas to numpy, and
x_test_numpy = x_test.to_numpy()
x_test_tensor = torch.from_numpy(x_test_numpy)
y_test_numpy = y_test.to_numpy()
y_test_tensor = torch.from_numpy(y_test_numpy)
print(x_test_tensor, y_test_tensor)

In [ ]:
y_train_tensor = y_train_tensor.unsqueeze(1)
y_test_tensor = y_test_tensor.unsqueeze(1)

In [ ]:
y_train_tensor

In [ ]:
y_test_tensor

In [ ]:
train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)

In [ ]:
test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)

In [ ]:
test_dataset[0]

In [ ]:
training_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
for x_test, y_test in validation_loader:
    # Forward pass only to get logits/output
    outputs = model(x_test.float())
    #print(outputs)
    #print(y_test[0])

In [ ]:
# Display image and label.
#x_test, y_test = validation_loader
##print(f"Feature batch shape: {x_test_features.size()}")
#print(f"Labels batch shape: {y_test_labels.size()}")
##img = x_test[0].squeeze()
##label = y_test[0]
##plt.imshow(img, cmap="gray")
##plt.show()
##print(f"Label: {label}")

In [ ]:
y_test
#y_test
#x_test

In [ ]:
''' STEP 7: TRAIN THE MODEL '''
iter = 0
for epoch in range(num_epochs):
    for x_train, y_train in training_loader:
        optimizer.zero_grad()
        # Forward pass to get output/logits
        outputs = model(x_train.float())   
        #outputs = model(x_test[0].float())
    #print(outputs)
        #(outputs)     
        # Calculate Loss: softmax --> cross entropy loss
        loss = criterion(outputs, y_train.float())
        # Getting gradients w.r.t. parameters
        loss.backward()
        # Updating parameters
        optimizer.step()        
        iter += 1
        if iter % 500 == 0:
            # Calculate Accuracy         
            correct = 0
            total = 0
            # Iterate through test dataset
            #example =  x_test
            for x_test, y_test in validation_loader:
                # Forward pass only to get logits/output
                outputs = model(x_test.float())
                m = nn.Sigmoid()
                #input = torch.randn(2)
                Normalized_output = m(outputs)
                #print( Normalized_output)
                predictions = Normalized_output > 0.5
                predictions_integer = predictions.long()
                #print(predictions)
                #print(outputs)
                # Get predictions from the maximum value
                #_, predicted = torch.max(outputs.data, 1)
                #print(outputs.data, 1)
                # Total number of labels
                total += y_test.size(0)  
                #print(total)              
                # Total correct predictions
                correct += (predictions_integer == y_test).sum()
                #print(correct)
            accuracy = 100 * correct / total
            #print(total)
            #print(correct)
            #print(accuracy)
            # Print Loss
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))